In [1]:
from example.nuscenes.av_discretizer  import AVDiscretizer
from example.nuscenes.transition import TransitionRecorded
from nuscenes import NuScenes
from pathlib import Path
import pgeon.policy_graph as PG
from example.nuscenes.environment import SelfDrivingEnvironment


In [2]:
# This is the path where you stored your copy of the nuScenes dataset.

#MINI
DATAROOT = Path('/home/saramontese/Desktop/MasterThesis/example/nuscenes/dataset/data/sets/nuscenes')
nuscenes = NuScenes('v1.0-mini', dataroot=DATAROOT)


#FULL
#DATAROOT = Path('/media/saramontese/Riccardo 500GB/NuScenesDataset/data/sets/nuscenes')
#nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.451 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


# Load the CSV data into a DataFrame


In [3]:

dtype_dict = {
    'modality': 'category',  # for limited set of modalities, 'category' is efficient
    'scene_token': 'str',  
    'timestamp': 'str',  # To enable datetime operations
    'rotation': 'object',  # Quaternion (lists)
    'x': 'float64',
    'y': 'float64',
    'z': 'float64',
    'yaw': 'float64',  
    'velocity': 'float64',
    'acceleration': 'float64',
    'heading_change_rate': 'float64',
    'delta_local_x': 'float64',
    'delta_local_y': 'float64'
}
df = pd.read_csv(DATAROOT / 'dataset_from_ego.csv', dtype=dtype_dict, parse_dates=['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'])




df.head()

,modality,scene_token,timestamp,rotation,x,y,z,yaw,velocity,acceleration,heading_change_rate,delta_local_x,delta_local_y
0,lidar,2fc3753772e241f2ab2cd16a784cc680,2018-08-30 19:25:09.301132,"[-0.1557237496690409, 0.011222825001502285, 0....",329.647494,660.196689,0.0,2.828707,5.108550,-0.034804,-0.045330,0.000000,0.000000
1,lidar,2fc3753772e241f2ab2cd16a784cc680,2018-08-30 19:25:09.801064,"[-0.16117694469754024, 0.009157178788916257, 0...",327.233834,661.025514,0.0,2.817706,5.104695,-0.007711,-0.022006,0.045648,2.551592
2,lidar,2fc3753772e241f2ab2cd16a784cc680,2018-08-30 19:25:10.200261,"[-0.16222999791894513, 0.010261981974878865, -...",325.362312,661.686833,0.0,2.815673,4.972301,-0.331650,-0.005091,0.031316,1.984681
3,lidar,2fc3753772e241f2ab2cd16a784cc680,2018-08-30 19:25:10.651414,"[-0.16605361559690043, 0.007585584174464603, -...",323.362925,662.405363,0.0,2.807968,4.709219,-0.583132,-0.017080,0.040541,2.124192
4,lidar,2fc3753772e241f2ab2cd16a784cc680,2018-08-30 19:25:11.149672,"[-0.1790782272855317, 0.009448521999393368, -0...",321.358449,663.164911,0.0,2.781689,4.302103,-0.817080,-0.052741,0.061262,2.142681


In [4]:
df.shape

(384, 13)

## All scenes trajectory information


Now, apply the above to all scenes in mini dataset.

In [21]:
av_discretizer = AVDiscretizer()
trajectory = av_discretizer.compute_trajectory(df)

State 0: [0.0, 0.0, 5.108549775006556, 2.8287073423413176, -0.0453298240761698, -0.0348037318593867, Timestamp('2018-08-30 19:25:09.301132')]
Discretized state: 0 [(82, 165)]&[<Velocity.LOW: 2>]&[<Rotation.RIGHT: 5>]
Action: {<Action.STRAIGHT: 11>}

State 1: [0.0456478454936385, 2.5515920468546924, 5.10469490440442, 2.817705897134056, -0.0220058832146411, -0.007710789871694, Timestamp('2018-08-30 19:25:09.801064')]
Discretized state: 1 [(81, 165)]&[<Velocity.LOW: 2>]&[<Rotation.RIGHT: 5>]
Action: {<Action.STRAIGHT: 11>}

State 2: [0.0313156638906143, 1.9846806565211763, 4.972301147669266, 2.815673479305952, -0.0050912652853189, -0.3316501795733787, Timestamp('2018-08-30 19:25:10.200261')]
Discretized state: 2 [(81, 165)]&[<Velocity.LOW: 2>]&[<Rotation.RIGHT: 5>]
Action: {<Action.STRAIGHT: 11>, <Action.BRAKE: 5>}

State 3: [0.0405408994690575, 2.1241916563087795, 4.709219465049271, 2.807967705113446, -0.0170801794347065, -0.5831318479983395, Timestamp('2018-08-30 19:25:10.651414')]
Disc

In [22]:

transition = TransitionRecorded()
transition.process_and_save_transitions(trajectory, av_discretizer.unique_states, path='/home/saramontese/Desktop/MasterThesis/example/nuscenes/dataset/data/sets/nuscenes/')

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
504
506
508
510
512
514
516
518
520
522
524
526

# Policy Graph Generation

In [23]:
path_pg_data = Path('/home/saramontese/Desktop/MasterThesis/example/nuscenes/dataset/data/sets/nuscenes')
env = SelfDrivingEnvironment(0,0,0,0)


/home/saramontese/.local/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [24]:
av_discretizer = AVDiscretizer()

pg = PG.PolicyGraph.from_nodes_and_edges(str(path_pg_data / 'nuscenes_nodes.csv'), str(path_pg_data / 'nuscenes_edges.csv'), env, av_discretizer  )

In [25]:
print(f'Number of nodes: {len(pg.nodes)}')
print(f'Number of edges: {len(pg.edges)}')

Number of nodes: 235
Number of edges: 284


In [26]:
arbitrary_state = list(pg.nodes)[0]

print(arbitrary_state)
print(f'  Times visited: {pg.nodes[arbitrary_state]["frequency"]}')
print(f'  p(s):          {pg.nodes[arbitrary_state]["probability"]:.3f}')

((82, 165), <Velocity.LOW: 2>, <Rotation.RIGHT: 5>)
  Times visited: 1
  p(s):          0.003


In [27]:
arbitrary_edge = list(pg.edges)[0]

print(f'From:    {arbitrary_edge[0]}')
print(f'Action:  {arbitrary_edge[2]}')
print(f'To:      {arbitrary_edge[1]}')
print(f'  Times visited:      {pg[arbitrary_edge[0]][arbitrary_edge[1]][arbitrary_edge[2]]["frequency"]}')
print(f'  p(s_to,a | s_from): {pg[arbitrary_edge[0]][arbitrary_edge[1]][arbitrary_edge[2]]["probability"]:.3f}')

From:    ((82, 165), <Velocity.LOW: 2>, <Rotation.RIGHT: 5>)
Action:  5
To:      ((81, 165), <Velocity.LOW: 2>, <Rotation.RIGHT: 5>)
  Times visited:      1
  p(s_to,a | s_from): 1.000


In [28]:
len(pg._trajectories_of_last_fit)


0